# The Variational Quantum Eigensolver

This notebook was originally prepared by Ryan LaRose for the Spring 2019 QuIC seminar using Pyquil. Justin Lane updated it to run using qiskit for the Fall 2019 QuIC seminar.

Note also that the Qiskit aqua package has VQE build into it. In fact, the Qiskit tutorial github, which may be found [here](https://github.com/Qiskit/qiskit-iqx-tutorials), has a notebook specifically dedicated to solving this exact problem. However, we'll stray away from using that in favor of a more low level approach so that we can put the variational circuit principles we've learned to use.

<img src="http://nobel.scas.bcit.ca/chem0010/unit4/images/h2.gif" alt="H2" style="width:300px;height:300px;">

Now that we've learned how to implement variational algorithms using Qiskit, we'd like to use our knowledge to solve a real problem! It turns out that, as demonstrated in [1](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.6.031007) which we'll be following for this notebook, it's possible to map the hydrogen molecule onto a 2 qubit system and find the energy landscape via an algorithm called the **variation quantum eigensolver** (VQE).

In [1]:
"""Imports for the notebook."""
import numpy as np
from numpy.random import random
from scipy.optimize import minimize
import matplotlib.pyplot as plt


import qiskit as qis

%matplotlib inline

In [2]:
"""Check Qiskit version"""
qis.qiskit.__version__

'0.9.0'

# The Hamiltonian for $H_2$

We can write down the Hamiltonian for $H_2$ in the following form [1]

\begin{equation}
    H = g_0 I + g_1 Z_0 + g_2 Z_1 + g_3 Z_0 Z_1 + g_4 X_0 X_1 + g_5 Y_0 Y_1
\end{equation}

where $g_i$ are numerical values that depend on the bond length $R$ between Hydrogren molecules. This writing of the Hamiltonian:

1. Uses the minimal [STO-6G basis](https://en.wikipedia.org/wiki/STO-nG_basis_sets),
1. Uses the Bravyi-Kitaev transform, and
1. Reduces resources (qubit number) by symmetry considerations (see paper).

Each coefficient $g_i$ is a function of bond length, and is specified below, where the first column is the bond length $R$:

In [3]:
"""The organization of the numeric values in the table below is

RADIUS  | g0    | g1    | g2    | g3    | g4    | g5

where it is understood each g is a function of the RADIUS R.

Source: Table 1 of Appendix C of [1].
"""

table = np.array([
        [0.2000, 2.8489, 0.5678, -1.4508, 0.6799, 0.07910, 0.07910],
        [0.2500, 2.1868, 0.5449, -1.2870, 0.6719, 0.07980, 0.07980],
        [0.3000, 1.7252, 0.5215, -1.1458, 0.6631, 0.08060, 0.08060],
        [0.3500, 1.3827, 0.4982, -1.0226, 0.6537, 0.08150, 0.08150],
        [0.4000, 1.1182, 0.4754, -0.9145, 0.6438, 0.08250, 0.08250],
        [0.4500, 0.9083, 0.4534, -0.8194, 0.6336, 0.08350, 0.08350],
        [0.5000, 0.7381, 0.4325, -0.7355, 0.6233, 0.08460, 0.08460],
        [0.5500, 0.5979, 0.4125, -0.6612, 0.6129, 0.08580, 0.08580],
        [0.6000, 0.4808, 0.3937, -0.5950, 0.6025, 0.08700, 0.08700],
        [0.6500, 0.3819, 0.3760, -0.5358, 0.5921, 0.08830, 0.08830],
        [0.7000, 0.2976, 0.3593, -0.4826, 0.5818, 0.08960, 0.08960],
        [0.7500, 0.2252, 0.3435, -0.4347, 0.5716, 0.09100, 0.09100],
        [0.8000, 0.1626, 0.3288, -0.3915, 0.5616, 0.09250, 0.09250],
        [0.8500, 0.1083, 0.3149, -0.3523, 0.5518, 0.09390, 0.09390],
        [0.9000, 0.06090, 0.3018, -0.3168, 0.5421, 0.09540, 0.09540],
        [0.9500, 0.01930, 0.2895, -0.2845, 0.5327, 0.09700, 0.09700],
        [1.000, -0.01720, 0.2779, -0.2550, 0.5235, 0.09860, 0.09860],
        [1.050, -0.04930, 0.2669, -0.2282, 0.5146, 0.1002, 0.1002],
        [1.100, -0.07780, 0.2565, -0.2036, 0.5059, 0.1018, 0.1018],
        [1.150, -0.1029, 0.2467, -0.1810, 0.4974, 0.1034, 0.1034],
        [1.200, -0.1253, 0.2374, -0.1603, 0.4892, 0.1050, 0.1050],
        [1.250, -0.1452, 0.2286, -0.1413, 0.4812, 0.1067, 0.1067],
        [1.300, -0.1629, 0.2203, -0.1238, 0.4735, 0.1083, 0.1083],
        [1.350, -0.1786, 0.2123, -0.1077, 0.4660, 0.1100, 0.1100],
        [1.400, -0.1927, 0.2048, -0.09290, 0.4588, 0.1116, 0.1116],
        [1.450, -0.2053, 0.1976, -0.07920, 0.4518, 0.1133, 0.1133],
        [1.500, -0.2165, 0.1908, -0.06660, 0.4451, 0.1149, 0.1149],
        [1.550, -0.2265, 0.1843, -0.05490, 0.4386, 0.1165, 0.1165],
        [1.600, -0.2355, 0.1782, -0.04420, 0.4323, 0.1181, 0.1181],
        [1.650, -0.2436, 0.1723, -0.03420, 0.4262, 0.1196, 0.1196],
        [1.700, -0.2508, 0.1667, -0.02510, 0.4204, 0.1211, 0.1211],
        [1.750, -0.2573, 0.1615, -0.01660, 0.4148, 0.1226, 0.1226],
        [1.800, -0.2632, 0.1565, -0.00880, 0.4094, 0.1241, 0.1241],
        [1.850, -0.2684, 0.1517, -0.00150, 0.4042, 0.1256, 0.1256],
        [1.900, -0.2731, 0.1472, 0.005200, 0.3992, 0.1270, 0.1270],
        [1.950, -0.2774, 0.1430, 0.01140, 0.3944, 0.1284, 0.1284],
        [2.000, -0.2812, 0.1390, 0.01710, 0.3898, 0.1297, 0.1297],
        [2.050, -0.2847, 0.1352, 0.02230, 0.3853, 0.1310, 0.1310],
        [2.100, -0.2879, 0.1316, 0.02720, 0.3811, 0.1323, 0.1323],
        [2.150, -0.2908, 0.1282, 0.03170, 0.3769, 0.1335, 0.1335],
        [2.200, -0.2934, 0.1251, 0.03590, 0.3730, 0.1347, 0.1347],
        [2.250, -0.2958, 0.1221, 0.03970, 0.3692, 0.1359, 0.1359],
        [2.300, -0.2980, 0.1193, 0.04320, 0.3655, 0.1370, 0.1370],
        [2.350, -0.3000, 0.1167, 0.04650, 0.3620, 0.1381, 0.1381],
        [2.400, -0.3018, 0.1142, 0.04950, 0.3586, 0.1392, 0.1392],
        [2.450, -0.3035, 0.1119, 0.05230, 0.3553, 0.1402, 0.1402],
        [2.500, -0.3051, 0.1098, 0.05490, 0.3521, 0.1412, 0.1412],
        [2.550, -0.3066, 0.1078, 0.05720, 0.3491, 0.1422, 0.1422],
        [2.600, -0.3079, 0.1059, 0.05940, 0.3461, 0.1432, 0.1432],
        [2.650, -0.3092, 0.1042, 0.06140, 0.3433, 0.1441, 0.1441],
        [2.700, -0.3104, 0.1026, 0.06320, 0.3406, 0.1450, 0.1450],
        [2.750, -0.3115, 0.1011, 0.06490, 0.3379, 0.1458, 0.1458],
        [2.800, -0.3125, 0.09970, 0.06650, 0.3354, 0.1467, 0.1467],
        [2.850, -0.3135, 0.09840, 0.06790, 0.3329, 0.1475, 0.1475]
])

To speed up the process, we'll reduce the number of bond lengths we consider.

In [4]:
"""Reduce the number of bond lengths considered."""
# only consider the first N bond lengths, and increase the spacing
N = 50
step = 2

# grab the reduced coeffs and bond lengths
table = table[0:N:step]
radii = table[:, 0]
coeffs = table[:, 1:]

# show the reduced set of coefficients at each bond length
print("Bond lengths (Angstroms); Coefficients (Hartree):")

for i in range(len(table)):
    print(radii[i],";",coeffs[i])

Bond lengths (Angstroms); Coefficients (Hartree):
0.2 ; [ 2.8489  0.5678 -1.4508  0.6799  0.0791  0.0791]
0.3 ; [ 1.7252  0.5215 -1.1458  0.6631  0.0806  0.0806]
0.4 ; [ 1.1182  0.4754 -0.9145  0.6438  0.0825  0.0825]
0.5 ; [ 0.7381  0.4325 -0.7355  0.6233  0.0846  0.0846]
0.6 ; [ 0.4808  0.3937 -0.595   0.6025  0.087   0.087 ]
0.7 ; [ 0.2976  0.3593 -0.4826  0.5818  0.0896  0.0896]
0.8 ; [ 0.1626  0.3288 -0.3915  0.5616  0.0925  0.0925]
0.9 ; [ 0.0609  0.3018 -0.3168  0.5421  0.0954  0.0954]
1.0 ; [-0.0172  0.2779 -0.255   0.5235  0.0986  0.0986]
1.1 ; [-0.0778  0.2565 -0.2036  0.5059  0.1018  0.1018]
1.2 ; [-0.1253  0.2374 -0.1603  0.4892  0.105   0.105 ]
1.3 ; [-0.1629  0.2203 -0.1238  0.4735  0.1083  0.1083]
1.4 ; [-0.1927  0.2048 -0.0929  0.4588  0.1116  0.1116]
1.5 ; [-0.2165  0.1908 -0.0666  0.4451  0.1149  0.1149]
1.6 ; [-0.2355  0.1782 -0.0442  0.4323  0.1181  0.1181]
1.7 ; [-0.2508  0.1667 -0.0251  0.4204  0.1211  0.1211]
1.8 ; [-0.2632  0.1565 -0.0088  0.4094  0.1241  0.1241

If you worked your way through the Variational Algorithms notebook, this should look familiar: we have a Hamiltonian written as a sum of Pauli operators with weights.

<font size=8 color="#009600">&#9998;</font> **Do this:** Make a list of Pauli operators to simulate the $H_2$ Hamiltonian.

In [ ]:
# Your code here


The circuit for preparing the Ansatz wavefunction can be found in Figure (1) of [1], as well as the logic for arriving to that circuit. Here, I've prepared a function that takes in the variational parameter ```theta``` and spits out the quantum circuit to prepare the Ansatz wavefunction:

In [5]:
def VQE_circuit(theta: float): 
    # Get a circuit and registers
    qreg = qis.QuantumRegister(2)
    creg = qis.ClassicalRegister(2)
    circ = qis.QuantumCircuit(qreg, creg)

    # Add gates with particular parameters
    circ.x(0)
    
    circ.rx(-np.pi/2, qreg[0])
    circ.ry(np.pi/2, qreg[1])
    
    circ.cx(qreg[1], qreg[0])
    circ.rz(theta, qreg[0])
    circ.cx(qreg[1], qreg[0])
    
    circ.rx(np.pi/2, qreg[0])
    circ.ry(-np.pi/2, qreg[1])
    
    return circ

my_circ = VQE_circuit(0.4)
print(my_circ)

            ┌───┐    ┌───────────┐┌───┐┌─────────┐┌───┐ ┌──────────┐
q0_0: |0>───┤ X ├────┤ Rx(-pi/2) ├┤ X ├┤ Rz(0.4) ├┤ X ├─┤ Rx(pi/2) ├
         ┌──┴───┴───┐└───────────┘└─┬─┘└─────────┘└─┬─┘┌┴──────────┤
q0_1: |0>┤ Ry(pi/2) ├───────────────■───────────────■──┤ Ry(-pi/2) ├
         └──────────┘                                  └───────────┘
 c0_0: 0 ═══════════════════════════════════════════════════════════
                                                                    
 c0_1: 0 ═══════════════════════════════════════════════════════════
                                                                    


I'll also copy over the expectation value calculator function we developed in the other notebook. As a reminder, this takes in a circuit and a string of Pauli operators and calculates the expectation value of that string.

In [6]:
def expectation(circuit: qis.QuantumCircuit, pauli_string: str, shots: int = 10000) -> float:
    """Returns the expectation value of the pauli string in the state prepared by the circuit."""
    
    temp = circuit.copy()
    ident = []
    
    ### Your code here!
    if len(circuit.qregs) != 1:
        raise ValueError("Circuit should have only one quantum register.")
    if len(circuit.cregs) != 1:
        print("# cregs =", len(circuit.cregs))
        raise ValueError("Circuit should have only one classical register.")
    
    qreg = circuit.qregs[0]
    creg = circuit.cregs[0]
    nqubits = len(qreg)
    pauli_string = pauli_string.upper().strip()
    
    if len(pauli_string) != nqubits:
        raise ValueError(
            f"Circuit has {nqubits} qubits but pauli_string has {len(pauli_string)} operators."
        )
    
    for (qubit, pauli) in enumerate(pauli_string):
        if pauli == "I":
            ident.append(1)
            continue
        elif pauli == "X":
            ident.append(0)
            temp.h(qreg[qubit])
            temp.measure(qreg[qubit], creg[qubit])
        elif pauli == "Y":
            ident.append(0)
            temp.s(qreg[qubit])
            temp.h(qreg[qubit])
            temp.measure(qreg[qubit], creg[qubit])
        elif pauli == "Z":
            ident.append(0)
            temp.measure(qreg[qubit], creg[qubit])
        else:
            raise ValueError(f"{pauli} is an invalid Pauli string key. Should be I, X, Y, or Z.") 
    
    if all(ident) == True:
        return 1 # If all paulis are I, <I> = 1. Also suppresses a super annoying warning.
    
    
    # Execute the circuit
    backend = qis.BasicAer.get_backend("qasm_simulator")
    job = qis.execute(temp, backend, shots=shots)
    counts = job.result().get_counts()
    
    # Do the postprocessing
    val = 0.0
    for bitstring, count in counts.items():
        sign = (-1) ** bitstring.count("0")
        val += sign * count
    
    return val / shots    

<font size=8 color="#009600">&#9998;</font> **Do this:** As an exercise to yourself, in the following cell write a function which takes in a list of Pauli operators, a list of weights for those operators, and returns the expectation value taken in an input circuit. We did this in the other notebook, but it's good to remember exactly what's going on!

In [7]:
"""Function to compute the cost of any Hamiltonian in the state prepared by the circuit."""
def cost(circuit, weights, paulis):
    """Returns <psi|H|psi> where |psi> is prepared by the circuit
    and the weights and paulis define a Hamiltonian H.
    
    Args:
        circuit: Circuit which prepares a state.
        weights: List of floats which are the coeffs/weights of each Pauli string.
        paulis:  List of strings which specify the Paulis.
    """
    if len(weights) != len(paulis):
        raise ValueError("Args weights and paulis must have the same length.")
    ### Your code here!
    

Now, all the pieces are in place: we can, given our list of Hamiltonian parameters ```table``` and some variational parameter ```theta```, compute the energy of the $H_2$ at a given radius.

<font size=8 color="#009600">&#9998;</font> **Do this:** Use the function you wrote to calculate the energy of the molecular hydrogen at $R = 0.2$ Angstroms for variational parameter ```theta``` = 0 

In [8]:
"""Calculate the energy of molecular hydrogen!"""

# Your code here


'Calculate the energy of molecular hydrogen!'

# Minimizing the Total Energy

Just like we did in the Variational algorithms notebook, we can now define an objective function that we want to minimize. The objective function is just a function of our parameter $\theta$ that we are trying to minimize. Remember, this is only for a given radii index `r`: we'll have to loop over radii to get the full energy landscape.

In [9]:
def obj(theta):
    en = cost(VQE_circuit(0), weights, H2_paulis)
    print("Current energy:", en, end="\r")
    return en

We now use a classical optimization algorithm to minimize the objective function:

In [ ]:
result = minimize(obj, x0=[0], method="Powell")

In [ ]:
"""See the optimization results."""
print("Lowest cost function value found:", result.fun)
print("Best parameters:", result.x)

# Computing the Potential Energy Surface

Now, we can loop through all bond lengths and minimize the total energy at each one. Since we haven't seen our bond lengths or coefficients in a while, we'll print them out here.

In [ ]:
"""Don't forget the bond lengths and coefficients!"""

# show the reduced set of coefficients at each bond length
print("Bond lengths (Angstroms); Coefficients (Hartree):")

for i in range(len(table)):
    print(radii[i],";",coeffs[i])

## Warning

I'm not an expert at what's going on behind the scenes is qiskit, but I think the way the code is written right now may be opening a new instance of `my_circ` every time we calculate the expectation value. The code below will optimize `theta` for *every* bond length, and can open upwards of 10,000 instances. This will take a while... and it may crash the notebook. If anyone can figure out how to not have this happen, you're smarter than me. Best of luck?

In [ ]:
"""Compute the minimum energy at each bond length."""
energies = []
for r in range(len(radii)):
    # status update
    print("Current bond length:", radii[r], "Angstroms.")
    
    # define the objective function for this radius
    obj = lambda theta: cost(VQE_circuit(theta), coeffs[r], H2_paulis)
    
    # compute the minimum energy
    opt = minimize(obj, x0=[0], method="Powell")
    
    # append the minimum energy to the list
    energies.append(opt.fun)
    print("Energy at r =",radii[r],'Angstroms =',opt.fun,'Hartree')

# Plotting the Potential Energy Surface
We can now visualize the potential energy surface by plotting energies (in Hartree) at each bond length (in Angstroms).

In [ ]:
"""Plot the potential energy surface."""
plt.plot(radii, energies, "-o", linewidth=2)
plt.grid()
plt.title("H2 Potential Energy Surface")
plt.xlabel("Bond Length [Angstroms]")
plt.ylabel("Energy [Hartree]")

If everything goes well, you should get something that looks like a Lennard-Jones potential. Congratulations! You just implemented a quantum algorithm to do a quantum simulation!

JL would like to reiterate that this notebook was mostly written by Ryan LaRose. Many thanks to Yousif Almulla for translating the coefficients into code and providing an initial program.

# References

[[1] P. J. J. O’Malley et al., “Scalable Quantum Simulation of Molecular Energies,” 
Physical Review X, vol. 6, no. 3, Jul. 2016.](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.6.031007)
